In [1]:
%matplotlib inline

In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
import tensorflow as tf

2024-12-28 10:55:59.129069: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout, Concatenate
from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# load_data() dataset splits the dataset to training and testing

In [7]:
data = load_data()
((attr_train, tgt_train), (attr_test, tgt_test)) = data

In [8]:
attr_test.shape

(10000, 28, 28)

In [9]:
attr_train.shape

(60000, 28, 28)

In [10]:
attr_train.reshape(60000, -1).shape

(60000, 784)

In [11]:
input_layer = Input((28, 28))

In [12]:
input_layer

<KerasTensor: shape=(None, 28, 28) dtype=float32 (created by layer 'input_1')>

In [13]:
flatten_layer = Flatten()(input_layer)

In [14]:
flatten_layer

<KerasTensor: shape=(None, 784) dtype=float32 (created by layer 'flatten')>

In [15]:
layer1 = Dense(64, activation='relu')(flatten_layer)
layer2 = Dense(128, activation='relu')(layer1)
layer3 = Dense(64, activation='relu')(layer2)
output_layer = Dense(10, activation='softmax')(layer3)

In [16]:
output_layer

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_3')>

In [17]:
model = Model(inputs=input_layer, outputs=output_layer)

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 64)                50240     
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 67,466
Trainable params: 67,466
Non-trainable p

In [19]:
block_beginning = Dense(20, activation='relu')(flatten_layer)

block_a1 = Dense(11, activation='relu')(block_beginning)
block_a2 = Dense(20, activation='relu')(block_a1)
block_a3 = Dense(11, activation='relu')(block_a2)

block_b = Dense(20, activation='relu')(block_beginning)
block_c = Dense(30, activation='relu')(block_beginning)

block_end = Concatenate()([block_a3, block_b, block_c])

In [20]:
block = Model(inputs=block_beginning, outputs=block_end)

In [21]:
block.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 dense_5 (Dense)                (None, 11)           231         ['input_2[0][0]']                
                                                                                                  
 dense_6 (Dense)                (None, 20)           240         ['dense_5[1][0]']                
                                                                                                  
 dense_7 (Dense)                (None, 11)           231         ['dense_6[1][0]']                
                                                                                            

In [22]:
plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [23]:
np.array([-1, -1], dtype=int)

array([-1, -1])

In [24]:
final_layer = Dense(20, activation='relu')(block.output)

In [25]:
Model(inputs=block.input, outputs=final_layer).summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 dense_5 (Dense)                (None, 11)           231         ['input_2[0][0]']                
                                                                                                  
 dense_6 (Dense)                (None, 20)           240         ['dense_5[1][0]']                
                                                                                                  
 dense_7 (Dense)                (None, 11)           231         ['dense_6[1][0]']                
                                                                                            

In [26]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 64)                50240     
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 67,466
Trainable params: 67,466
Non-trainable p

In [27]:
# creating class for custom loss func
class DoubleClassificationLoss:
    def call(self, y_true, y_pred):
        return ... # the return statement of the loss function

In [28]:
# compile the nns
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
model.fit(attr_train, tgt_train, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 19s 9ms/step - loss: 1.2622 - accuracy: 0.7296
Epoch 2/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.5792 - accuracy: 0.8009
Epoch 3/3
1875/1875 [==============================] - 18s 9ms/step - loss: 0.5087 - accuracy: 0.8231


In [30]:
# checking if overfitting
model.evaluate(attr_test, tgt_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.4925 - accuracy: 0.8324


[0.49245527386665344, 0.8324000239372253]

In [31]:
# not overfitting

In [32]:
# applying dropout regularization
# it is applied after the training of the layer
# it is not recommended to apply dropout after first input layer

In [33]:
model_2 = Model(inputs=input_layer, outputs=output_layer)
model_2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model_2.fit(
    attr_train, 
    tgt_train, 
    epochs=3, 
    callbacks=[TensorBoard()]
)

Epoch 1/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.4664 - accuracy: 0.8367
Epoch 2/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.4149 - accuracy: 0.8518
Epoch 3/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.4083 - accuracy: 0.8543


In [34]:
model_2.evaluate(attr_test, tgt_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.4458 - accuracy: 0.8447


[0.4458167552947998, 0.8446999788284302]

In [40]:
# tensorboard is used to log the training of the model
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
